In [23]:
#Load Video
import cv2

# Load video file
video_path = 'videoplayback.mp4'  # Path to your video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video file")

In [25]:
#Frame Extraction
import os

output_dir = 'extracted_frames'
os.makedirs(output_dir, exist_ok=True)

frame_count = 0

# Extract frames from video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(output_dir, f'frame_{frame_count}.jpg')
    cv2.imwrite(frame_path, frame)
    frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames.")

Extracted 1109 frames.


In [27]:
#Spatio-Temporal Segmentation (using edge detection)
#Color Thresholding (Segmentation)
import cv2
import numpy as np

def color_thresholding(frame):
    # Convert frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define color range for thresholding (example: green objects)
    lower_bound = np.array([40, 40, 40])  # Lower HSV bound for green
    upper_bound = np.array([80, 255, 255])  # Upper HSV bound for green
    
    # Apply the threshold
    mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)
    
    # Perform bitwise-AND to extract the colored region
    result = cv2.bitwise_and(frame, frame, mask=mask)
    
    return mask, result

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if ret:
    mask, segmented_frame = color_thresholding(frame)
    cv2.imshow('Segmented Frame', segmented_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
cap.release()

In [29]:
#Object Tracking Across Frames (Using Contours and Centroids)
def track_objects(mask):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    tracked_objects = []
    
    for contour in contours:
        # Only consider contours with a significant area
        if cv2.contourArea(contour) > 500:
            # Calculate the centroid of the contour
            M = cv2.moments(contour)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                tracked_objects.append((cx, cy))
    
    return tracked_objects

# Example: Track objects on the segmented frame
tracked_objects = track_objects(mask)
print(f"Tracked Objects' Centroids: {tracked_objects}")

Tracked Objects' Centroids: [(113, 143), (616, 119), (506, 49), (399, 17), (186, 26)]


In [31]:
#Foreground vs. Background Segmentation (Using Background Subtraction)
# Initialize background subtractor
back_sub = cv2.createBackgroundSubtractorMOG2()

# Process all frames
cap = cv2.VideoCapture(video_path)
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fg_mask = back_sub.apply(frame)
    
    # Extract the segmented foreground objects
    mask, segmented_frame = color_thresholding(frame)
    
    # Combine foreground mask with color-based segmentation
    combined_mask = cv2.bitwise_and(fg_mask, mask)
    
    # Track objects on the combined mask
    tracked_objects = track_objects(combined_mask)
    
    # Visualize tracking by drawing centroids on the frame
    for cx, cy in tracked_objects:
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
    
    # Display the result
    cv2.imshow('Foreground Segmentation and Tracking', frame)
    
    if cv2.waitKey(30) & 0xFF == 27:  # Press ESC to exit
        break

    frame_idx += 1

cap.release()
cv2.destroyAllWindows()

In [33]:
#Scene Cut Detection (using histogram differences)
def detect_scene_cuts(hist_diff_threshold=0.5, soft_cut_window=5):
    scene_cuts = []
    
    # Initialize histograms for first frame
    prev_hist = None
    soft_cut_accum = 0
    
    for i in range(frame_count):
        frame_path = os.path.join(output_dir, f'frame_{i}.jpg')
        frame = cv2.imread(frame_path)
        
        if frame is None:
            continue
        
        # Calculate color histogram for the frame
        hist = cv2.calcHist([frame], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        
        if prev_hist is not None:
            # Calculate the correlation between histograms
            hist_diff = cv2.compareHist(prev_hist, hist, cv2.HISTCMP_CORREL)
            if hist_diff < hist_diff_threshold:  # Hard cut
                scene_cuts.append((i, "Hard Cut"))
                soft_cut_accum = 0  # Reset soft cut accumulator
            else:
                soft_cut_accum += 1
                if soft_cut_accum > soft_cut_window:  # Soft cut
                    scene_cuts.append((i, "Soft Cut"))
                    soft_cut_accum = 0
        
        prev_hist = hist
    
    return scene_cuts

scene_cuts = detect_scene_cuts()

# Output detected scene cuts
for cut in scene_cuts:
    print(f"Frame {cut[0]}: {cut[1]}")

Frame 6: Soft Cut
Frame 12: Soft Cut
Frame 18: Soft Cut
Frame 24: Soft Cut
Frame 30: Soft Cut
Frame 36: Soft Cut
Frame 42: Soft Cut
Frame 48: Soft Cut
Frame 54: Soft Cut
Frame 60: Soft Cut
Frame 66: Soft Cut
Frame 72: Soft Cut
Frame 78: Soft Cut
Frame 84: Soft Cut
Frame 90: Soft Cut
Frame 96: Soft Cut
Frame 102: Soft Cut
Frame 108: Soft Cut
Frame 114: Soft Cut
Frame 120: Soft Cut
Frame 126: Soft Cut
Frame 132: Soft Cut
Frame 138: Soft Cut
Frame 144: Soft Cut
Frame 150: Soft Cut
Frame 156: Soft Cut
Frame 162: Soft Cut
Frame 168: Soft Cut
Frame 174: Soft Cut
Frame 180: Soft Cut
Frame 186: Soft Cut
Frame 192: Soft Cut
Frame 198: Soft Cut
Frame 204: Soft Cut
Frame 210: Soft Cut
Frame 216: Soft Cut
Frame 222: Soft Cut
Frame 228: Soft Cut
Frame 234: Soft Cut
Frame 240: Soft Cut
Frame 246: Soft Cut
Frame 252: Soft Cut
Frame 258: Soft Cut
Frame 264: Soft Cut
Frame 270: Soft Cut
Frame 276: Soft Cut
Frame 282: Soft Cut
Frame 288: Soft Cut
Frame 294: Soft Cut
Frame 300: Soft Cut
Frame 306: Soft C

In [34]:
def visualize_scene_cuts(video_path, scene_cuts):
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # If this frame is a detected scene cut
        if frame_idx in scene_cuts:
            # Mark the frame with a red border or a label
            cv2.putText(frame, "Scene Cut", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
            frame = cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
        
        cv2.imshow('Scene Cut Detection', frame)
        
        if cv2.waitKey(30) & 0xFF == 27:  # Press ESC to exit
            break
        
        frame_idx += 1

    cap.release()
    cv2.destroyAllWindows()

visualize_scene_cuts('videoplayback.mp4', scene_cuts)

In [15]:
#Mark Scene Cuts (Visualize Results)
def visualize_scene_cuts(scene_cuts):
    for cut in scene_cuts:
        frame_idx, cut_type = cut
        frame_path = os.path.join(output_dir, f'frame_{frame_idx}.jpg')
        frame = cv2.imread(frame_path)
        
        if frame is not None:
            # Draw a red border around the frame to mark the scene cut
            marked_frame = cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
            
            # Display the frame with cut
            cv2.putText(marked_frame, cut_type, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
            cv2.imshow(f'Scene Cut - Frame {frame_idx}', marked_frame)
            cv2.waitKey(500)  # Display each frame for half a second
    
    cv2.destroyAllWindows()

visualize_scene_cuts(scene_cuts)

In [37]:
#Highlight Scene Cuts in Frames
import cv2

def highlight_scene_cuts(video_path, scene_cuts):
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    highlighted_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Check if the current frame is a detected scene cut
        if frame_idx in scene_cuts:
            # Add a red border to highlight the scene cut
            cv2.putText(frame, "Scene Cut", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
            frame = cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
            highlighted_frames.append(frame_idx)
        
        # Display the video with scene cut highlighting
        cv2.imshow('Scene Cut Detection', frame)
        
        # Break on ESC key press
        if cv2.waitKey(30) & 0xFF == 27:
            break
        
        frame_idx += 1

    cap.release()
    cv2.destroyAllWindows()

    return highlighted_frames

highlighted_frames = highlight_scene_cuts('videoplayback.mp4', scene_cuts)

In [39]:
#Summary of Scene Cut Boundaries
def generate_scene_cut_summary(scene_cuts):
    print("\n--- Scene Cut Summary ---")
    print(f"Total scene cuts detected: {len(scene_cuts)}")
    for idx, frame_num in enumerate(scene_cuts):
        print(f"Scene Cut {idx + 1}: Detected at frame {frame_num}")
    print("-------------------------")

#A summary of detected scene cuts
generate_scene_cut_summary(scene_cuts)


--- Scene Cut Summary ---
Total scene cuts detected: 187
Scene Cut 1: Detected at frame (6, 'Soft Cut')
Scene Cut 2: Detected at frame (12, 'Soft Cut')
Scene Cut 3: Detected at frame (18, 'Soft Cut')
Scene Cut 4: Detected at frame (24, 'Soft Cut')
Scene Cut 5: Detected at frame (30, 'Soft Cut')
Scene Cut 6: Detected at frame (36, 'Soft Cut')
Scene Cut 7: Detected at frame (42, 'Soft Cut')
Scene Cut 8: Detected at frame (48, 'Soft Cut')
Scene Cut 9: Detected at frame (54, 'Soft Cut')
Scene Cut 10: Detected at frame (60, 'Soft Cut')
Scene Cut 11: Detected at frame (66, 'Soft Cut')
Scene Cut 12: Detected at frame (72, 'Soft Cut')
Scene Cut 13: Detected at frame (78, 'Soft Cut')
Scene Cut 14: Detected at frame (84, 'Soft Cut')
Scene Cut 15: Detected at frame (90, 'Soft Cut')
Scene Cut 16: Detected at frame (96, 'Soft Cut')
Scene Cut 17: Detected at frame (102, 'Soft Cut')
Scene Cut 18: Detected at frame (108, 'Soft Cut')
Scene Cut 19: Detected at frame (114, 'Soft Cut')
Scene Cut 20: Dete

In [51]:
import os
import cv2

# Define the path to the directory where segmented frames will be saved
segmented_frames_dir = 'segmented_frames'  # Adjust the path as needed

# Check if the directory exists; if not, create it
if not os.path.exists(segmented_frames_dir):
    os.makedirs(segmented_frames_dir)
    print(f'Directory "{segmented_frames_dir}" created.')
else:
    print(f'Directory "{segmented_frames_dir}" already exists.')

def visualize_segmentation(frame_indices):
    for idx in frame_indices:
        seg_frame_path = os.path.join(segmented_frames_dir, f'segmented_frame_{idx}.jpg')
        seg_frame = cv2.imread(seg_frame_path)

        if seg_frame is not None:
            cv2.imshow(f'Segmented Frame {idx}', seg_frame)
            cv2.waitKey(0)  # Wait for a key press to close the image
        else:
            print(f'Frame {idx} not found at {seg_frame_path}')

# Example: Visualize segmentation for frames with scene cuts
cut_frame_indices = [cut[0] for cut in scene_cuts]
visualize_segmentation(cut_frame_indices)

# Clean up windows after displaying
cv2.destroyAllWindows()

Directory "segmented_frames" already exists.
